# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import sklearn.metrics as metrics

In [8]:
g0 = pd.read_csv(r"C:\Users\andre\Рабочий стол\Практикум\Специалист по Data Science Плюс. Часть 7\01 Машинное обучение в бизнесе\06 Самостоятельный проект\geo_data_0.csv").drop('id', axis = 1)
g1 = pd.read_csv(r"C:\Users\andre\Рабочий стол\Практикум\Специалист по Data Science Плюс. Часть 7\01 Машинное обучение в бизнесе\06 Самостоятельный проект\geo_data_1.csv").drop('id', axis = 1)
g2 = pd.read_csv(r"C:\Users\andre\Рабочий стол\Практикум\Специалист по Data Science Плюс. Часть 7\01 Машинное обучение в бизнесе\06 Самостоятельный проект\geo_data_2.csv").drop('id', axis = 1)


In [9]:
g0_features_train, g0_features_valid, g0_target_train, g0_target_valid = train_test_split(g0.drop('product', axis=1),g0['product'], test_size=0.25, random_state=42)
g1_features_train, g1_features_valid, g1_target_train, g1_target_valid = train_test_split(g1.drop('product', axis=1),g1['product'], test_size=0.25, random_state=42)
g2_features_train, g2_features_valid, g2_target_train, g2_target_valid = train_test_split(g2.drop('product', axis=1),g2['product'], test_size=0.25, random_state=42)

## Обучение и проверка модели

In [28]:
results = []
predictions = pd.DataFrame()

for i in range(3):
    model = LinearRegression()
    features_train = globals()[f'g{i}_features_train']
    target_train = globals()[f'g{i}_target_train']
    features_valid = globals()[f'g{i}_features_valid']
    target_valid = globals()[f'g{i}_target_valid']

    model.fit(pd.DataFrame(features_train), pd.Series(target_train))
    prediction = model.predict(pd.DataFrame(features_valid))

    mse = metrics.mean_squared_error(pd.Series(target_valid), prediction)
    r2 = metrics.r2_score(pd.Series(target_valid), prediction)
    mae = metrics.mean_absolute_error(pd.Series(target_valid), prediction)

    predictions[i] = prediction
    results.append([mse ** 0.5, r2, mae, prediction.mean()])


In [29]:
print(pd.DataFrame(results).rename(columns={0: 'mse', 1: 'r2', 2: 'mae', 3: 'predictions'}))

         mse        r2        mae  predictions
0  37.756600  0.272829  31.047682    92.398800
1   0.890280  0.999625   0.717012    68.712878
2  40.145872  0.196347  32.896959    94.771024


In [30]:
predictions.head()

,0,1,2
0,101.901017,0.844738,98.301916
1,78.217774,52.921612,101.592461
2,115.266901,135.110385,52.449099
3,105.618618,109.494863,109.922127
4,97.980185,-0.047292,72.411847


## Подготовка к расчёту прибыли

In [27]:
# Ключевые параметры для расчета прибыли
BUDGET = 10_000_000_000  # 10 млрд рублей
BARREL_PRICE = 450_000  # 450 тыс. рублей за 1000 баррелей
BEST_WELLS = 200  # Количество лучших скважин
TOTAL_WELLS = 500  # Всего исследуемых скважин

In [42]:
def calculate_profit():
    arr = []
    for i in range(3):
        selected = predictions[:][i].sort_values(ascending=False).loc[:BEST_WELLS]
        arr.append(selected.sum() * BARREL_PRICE)
        return arr


In [43]:
print(calculate_profit())


[354552809150.9114]


In [44]:
predictions[:][0].sort_values(ascending=False)

6958     176.536104
18194    176.274510
17251    173.249504
457      172.802708
2202     172.744977
            ...    
24951      9.986965
8613       9.867996
11880      8.503684
6966       4.600630
1586      -9.836371
Name: 0, Length: 25000, dtype: float64

## Расчёт прибыли и рисков 

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Выполнен шаг 1: данные подготовлены
- [ ]  Выполнен шаг 2: модели обучены и проверены
    - [ ]  Данные корректно разбиты на обучающую и валидационную выборки
    - [ ]  Модели обучены, предсказания сделаны
    - [ ]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [ ]  На экране напечатаны результаты
    - [ ]  Сделаны выводы
- [ ]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [ ]  Для всех ключевых значений созданы константы Python
    - [ ]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [ ]  По предыдущему пункту сделаны выводы
    - [ ]  Написана функция расчёта прибыли
- [ ]  Выполнен шаг 4: посчитаны риски и прибыль
    - [ ]  Проведена процедура *Bootstrap*
    - [ ]  Все параметры бутстрепа соответствуют условию
    - [ ]  Найдены все нужные величины
    - [ ]  Предложен регион для разработки месторождения
    - [ ]  Выбор региона обоснован